**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Jun 27 08:11:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 1.jpg
 2.jpg
 best.pt
'CLIL_DNS Rampon, Paccagnella, Serrentino.gdoc'
'Colab Notebooks'
'Configurazioni PC.gsheet'
 Homework2
 images.zip
 Istanbul
 models
'Modulo senza titolo.gform'
'My Drive'
 ProgettoComputerVision
'Reti di Calcolatori - Appunti.gdoc'
'Via degli Dei'
'Via degli Dei.gdoc'
 yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15424, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 15424 (delta 0), reused 1 (delta 0), pack-reused 15423
Receiving objects: 100% (15424/15424), 14.03 MiB | 24.06 MiB/s, done.
Resolving deltas: 100% (10365/10365), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 2000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "hand" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3/' > data/obj.data
!mkdir data/obj

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov3/dataset_v7.zip -d data/obj

Output streaming troncato alle ultime 5000 righe.
  inflating: data/obj/dataset_v7/CARDS_COURTYARD_T_B_frame_0804_jpg.rf.1799e9ee792fac5bb69b321374091933.jpg  
  inflating: data/obj/dataset_v7/CARDS_COURTYARD_T_B_frame_0804_jpg.rf.1799e9ee792fac5bb69b321374091933.txt  
  inflating: data/obj/dataset_v7/CARDS_COURTYARD_T_B_frame_0823_jpg.rf.e08f749bc6d3b4f38971a6dd7e8858ed.jpg  
  inflating: data/obj/dataset_v7/CARDS_COURTYARD_T_B_frame_0823_jpg.rf.e08f749bc6d3b4f38971a6dd7e8858ed.txt  
  inflating: data/obj/dataset_v7/CARDS_COURTYARD_T_B_frame_0858_jpg.rf.e9e757f7dca211cb13b1509d315fb2c2.jpg  
  inflating: data/obj/dataset_v7/CARDS_COURTYARD_T_B_frame_0858_jpg.rf.e9e757f7dca211cb13b1509d315fb2c2.txt  
  inflating: data/obj/dataset_v7/CARDS_COURTYARD_T_B_frame_0880_jpg.rf.428de3f96330073419925794178c54b9.jpg  
  inflating: data/obj/dataset_v7/CARDS_COURTYARD_T_B_frame_0880_jpg.rf.428de3f96330073419925794178c54b9.txt  
  inflating: data/obj/dataset_v7/CARDS_COURTYARD_T_B_frame_0911_jpg.rf

In [ ]:
import glob
images_list = glob.glob("data/obj/dataset_v7/*.jpg")
print(images_list)
print(len(images_list))

['data/obj/dataset_v7/256_jpg.rf.d22b468bb105ba2136dc56d3737bd94d.jpg', 'data/obj/dataset_v7/CARDS_COURTYARD_B_T_frame_2666_jpg.rf.f2d782093bdb1494f30f9683d839bd78.jpg', 'data/obj/dataset_v7/CARDS_LIVINGROOM_T_B_frame_0565_jpg.rf.adf5155ba7a303f033898fbd435c6bcd.jpg', 'data/obj/dataset_v7/CARDS_LIVINGROOM_H_S_frame_0614_jpg.rf.856cbfd457ba201b90ab986048d8469c.jpg', 'data/obj/dataset_v7/PUZZLE_COURTYARD_S_B_frame_0251_jpg.rf.ff21f07b718cb667cf95a601ddef86e5.jpg', 'data/obj/dataset_v7/CARDS_COURTYARD_T_B_frame_1661_jpg.rf.87d80ab4374a8423f19b47fe55c61100.jpg', 'data/obj/dataset_v7/177_jpg.rf.4c8bc78fa82a5da5d69553f9e2b4256b.jpg', 'data/obj/dataset_v7/JENGA_OFFICE_T_H_frame_1794_jpg.rf.85aad461bddece3b2134b5c25328001a.jpg', 'data/obj/dataset_v7/CARDS_OFFICE_S_B_frame_0640_jpg.rf.2db7c29638d4c3142b2b51bf2e757b3c.jpg', 'data/obj/dataset_v7/CHESS_OFFICE_S_B_frame_1334_jpg.rf.ed0e14fe647dee7f621b512e0258a3c0.jpg', 'data/obj/dataset_v7/PUZZLE_LIVINGROOM_H_S_frame_1760_jpg.rf.3c7710395f62cb2267

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-06-27 08:14:13--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  43.1MB/s    in 4.1s    

2022-06-27 08:14:17 (37.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Output streaming troncato alle ultime 5000 righe.
 total_bbox = 287467, rewritten_bbox = 0.058094 % 
]2;1926/2000: loss=0.4 hours left=0.3
 1926: 0.435497, 0.513905 avg loss, 0.001000 rate, 10.066357 seconds, 123264 images, 0.278611 hours left
Loaded: 0.000066 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.796428), count: 6, class_loss = 0.681673, iou_loss = 0.395141, total_loss = 1.076814 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.887337), count: 4, class_loss = 0.128672, iou_loss = 0.096859, total_loss = 0.225531 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000631, iou_loss = 0.000000, total_loss = 0.000631 
 total_bbox = 287477, rewritten_bbox = 0.058092 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.776623), count: 6, class_loss = 0.464700, iou_loss = 0.290082, total_loss = 0.754782 
v3 (m